In [1]:
import logging
import sys
import shutil
import numpy as np
import pandas as pd
import seaborn as sns

from blueetl.analysis import Analyzer
from blueetl.constants import *
from blueetl.utils import load_yaml

import sys
sys.path.append('..')
import cortex_etl as c_etl

In [2]:
sns.set(style="ticks", context="paper", font="Helvetica Neue",
        rc={"axes.labelsize": 7, "legend.fontsize": 6, "axes.linewidth": 0.6, "xtick.labelsize": 6, "ytick.labelsize": 6,
            "xtick.major.size": 2, "xtick.major.width": 0.5, "xtick.minor.size": 1.5, "xtick.minor.width": 0.3,
            "ytick.major.size": 2, "ytick.major.width": 0.5, "ytick.minor.size": 1.5, "ytick.minor.width": 0.3,
            "axes.titlesize": 7, "axes.spines.right": False, "axes.spines.top": False})

In [3]:
np.random.seed(0)

# analysis_config_file = "configs/sscx_calibration_mgfix/2-PfrTransfer/2-PfrTransfer-6-3rdConnectionRemaining.yaml"
# sim_ind = 0

analysis_config_file = "../configs/sscx_calibration_mgfix/8-FRDistributionTest/8-FRDistributionTest-0-Test.yaml"
sim_ind = 16

analysis_config = load_yaml(analysis_config_file)
analysis_config['output'] = analysis_config['output'] + "/" + analysis_config['extraction']['target']
        
a = Analyzer(analysis_config)
a.extract_repo()
a.calculate_features()

a = a.apply_filter()
c_etl.create_figure_dirs(a)

Have keys ({'desired_connected_fr_L1I', 'desired_unconnected_fr_L23I', 'desired_connected_fr_L5I', 'desired_unconnected_fr_L5I', 'desired_unconnected_fr_L4I', 'desired_connected_fr_L23E', 'desired_connected_fr_L4I', 'desired_unconnected_fr_L4E', 'desired_connected_proportion_of_invivo_frs', 'desired_connected_fr_L6I', 'desired_connected_fr_L6E', 'desired_unconnected_fr_L23E', 'desired_unconnected_fr_L1I', 'desired_connected_fr_L5E', 'desired_connected_fr_L23I', 'desired_connected_fr_L4E', 'desired_unconnected_fr_L6I', 'desired_unconnected_fr_L5E', 'desired_unconnected_fr_L6E', 'depol_stdev_mean_ratio'}) that are not defined for section: Run
Have keys ({'desired_connected_fr_L1I', 'desired_unconnected_fr_L23I', 'desired_connected_fr_L5I', 'desired_unconnected_fr_L5I', 'desired_unconnected_fr_L4I', 'desired_connected_fr_L23E', 'desired_connected_fr_L4I', 'desired_unconnected_fr_L4E', 'desired_connected_proportion_of_invivo_frs', 'desired_connected_fr_L6I', 'desired_connected_fr_L6E', 'de

In [4]:
import matplotlib.pyplot as plt
# print(a.features.by_gid.df)
neuron_class_groups = [['L1_INH'], ['L23_EXC', 'L23_INH'], ['L4_EXC', 'L4_INH'], ['L5_EXC', 'L5_INH'], ['L6_EXC', 'L6_INH']]

for _, simulation_row in a.repo.simulations.df.iterrows():
    print(simulation_row['simulation_id'], simulation_row['ca'], simulation_row['depol_stdev_mean_ratio'], simulation_row['desired_connected_proportion_of_invivo_frs'])
    simulation_features_by_gid = a.features.by_gid.df.reset_index().etl.q(simulation_id=simulation_row['simulation_id'])
    
#     fig, axes = plt.subplots(1, len(neuron_classes), figsize=(20, 2))
    non_zero_MFRs_by_nc = []
    MFRs_by_nc = []
    class_ind = 0
    for nc_class_group in neuron_class_groups:
#         a = []
#         b = []
        for neuron_class in nc_class_group:
            class_ind += 1
            MFRs = simulation_features_by_gid.etl.q(neuron_class=neuron_class).mean_firing_rates_per_second
            non_zero_MFRs = MFRs[MFRs > 0.0]
            
#             a.append(MFRs)
#             b.append(non_zero_MFRs)
#         MFRs_by_nc.append(a)
#         non_zero_MFRs_by_nc.append(b)
            
            
#             axes[class_ind].hist(non_zero_MFRs, bins=np.arange(0, 5, 0.2))
#     plt.show()
    break
    
#     if (simulation_row['simulation_id'] == 0):
#         break

1 1.05 0.2 0.3


In [6]:
# 'L1_PV', 'L1_SST', 'L1_Htr3a', 
# 'L1_EXC', 
neuron_classes = ['L1_INH', 'L23_EXC', 'L23_INH', 'L23_PV', 'L23_SST', 'L23_Htr3a', 'L4_EXC', 'L4_INH', 'L4_PV', 'L4_SST', 'L4_Htr3a', 'L5_EXC', 'L5_INH', 'L5_PV', 'L5_SST', 'L5_Htr3a', 'L6_EXC', 'L6_INH', 'L6_PV', 'L6_SST', 'L6_Htr3a']
sim = a.repo.simulations.df.etl.q(ca=1.05, desired_connected_proportion_of_invivo_frs=0.3, depol_stdev_mean_ratio=0.4).iloc[0]
filtered_df = a.features.by_gid.df.reset_index().etl.q(neuron_class=neuron_classes, simulation_id=sim['simulation_id'])
filtered_df.loc[:, 'layer'] = filtered_df.apply(lambda row: row['neuron_class'].split('_')[0], axis = 1)
filtered_df.loc[:, 'neuron_type'] = filtered_df.apply(lambda row: row['neuron_class'].split('_')[1], axis = 1)
spiking_filtered_df = filtered_df.drop(filtered_df[filtered_df.mean_firing_rates_per_second == 0.0].index)

/nvme/isbister/1401358/ipykernel_245651/1131366636.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[:, 'layer'] = filtered_df.apply(lambda row: row['neuron_class'].split('_')[0], axis = 1)
/nvme/isbister/1401358/ipykernel_245651/1131366636.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[:, 'neuron_type'] = filtered_df.apply(lambda row: row['neuron_class'].split('_')[1], axis = 1)


In [ ]:
import seaborn as sns
bw = .1
hue_order= ["EXC", "INH", "PV", "SST", "Htr3a"]
palette = {"EXC":c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES["L5_EXC"]['color'], 
           "INH":c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES["L5_INH"]['color'], 
           "PV":c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES["L5_PV"]['color'], 
           "SST": c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES["L5_SST"]['color'], 
           "Htr3a":c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES["L5_Htr3a"]['color']}
ylabel = 'MFR (spikes/s)'

plt.figure(figsize=(10, 3))
sns.violinplot(data=filtered_df, x="layer", y="mean_firing_rates_per_second", hue="neuron_type", hue_order=hue_order, palette=palette, scale='width', bw=bw, linewidth=0.01)
plt.gca().set_ylim([0.0, 8.0])
plt.gca().set_ylabel(ylabel)
plt.savefig(str(a.analysis_config['figures']) + "/MFR-Violin.pdf")
plt.show()

plt.figure(figsize=(10, 2))
sns.violinplot(data=spiking_filtered_df, x="layer", y="mean_firing_rates_per_second", hue="neuron_type", hue_order=hue_order, palette=palette, scale='width', bw=bw, linewidth=0.01)
plt.gca().set_ylim([0.0, 10.0])
plt.gca().set_ylabel(ylabel)
plt.savefig(str(a.analysis_config['figures']) + "/MFR-SpikingOnly-Violin.pdf")
plt.show()

In [ ]:
a.repo.neurons.df.etl.q(neuron_class=['L1_PV', 'L1_SST', 'L1_Htr3a'])